In [1]:
import pytesseract,io,gc
from PIL import Image
from wand.image import Image as wi
import gc
import os
import time
import re
import pandas as pd
import time

In [2]:
pytesseract.pytesseract.tesseract_cmd = r'C:\Users\rmaci\AppData\Local\Tesseract-OCR\tesseract.exe'
def Image2Text(pdf_path):
    pdf=wi(filename=pdf_path,resolution=300)                                                                                                                
    pdfImg=pdf.convert('jpeg')                                                                                                                                                                              
    imgBlobs=[]
    extracted_text=[]
    try:        
        for img in pdfImg.sequence:
            page=wi(image=img)
            imgBlobs.append(page.make_blob('jpeg'))
            for i in range(0,5):
                [gc.collect() for i in range(0,10)]

        for imgBlob in imgBlobs:
            im=Image.open(io.BytesIO(imgBlob))
            text=pytesseract.image_to_string(im,lang='spa')
            text = text.replace(r"\n", " ")
            extracted_text.append(text)
            for i in range(0,5):
                [gc.collect() for i in range(0,10)]
        return (''.join([i.replace("\n"," ").replace("\n\n"," ") for i in extracted_text]))
        [gc.collect() for i in range(0,10)]
    finally:
        [gc.collect() for i in range(0,10)]
        img.destroy()

In [3]:
frases = ['derechos humanos', 'derechos humano', 'Derecho a una vida libre de violencia', 'Igualdad entre mujeres y hombres', 'Igualdad entre hombres y mujeres', 'personas con discapacidad',
 'pueblos indígenas', 'comunidades indígenas', 'personas mayores', 'personas de la tercera edad', 'niñez', 'niñas, niños', 'adolescentes']

dir = os.getcwd()
dir = os.path.join(dir, 'CNDH')
dir

'c:\\Users\\rmaci\\Documents\\Python\\3.8\\CNDH'

In [4]:
df = pd.read_csv('./Results/imgChecks Bando.csv')
df = df[df['Type'] == 'Scanned']['Title']
images = df.tolist()

In [5]:
dic = {
    "Title": [],
    "Page": [],
    "Phrase": []
}
dic

errorsDic = []

start_time = time.time()

for foldername, subfolders, files in os.walk(os.path.join(dir)):
    for file in files:
        if file in images:
            # try:
                print(file)
                text = Image2Text(os.path.join(foldername, file))
                if re.search('derechos humanos', text, re.IGNORECASE):
                    print('Pattern ' + frase.upper() + ' FOUND!')
                    dic['Title'].append(file)
                    dic['Page'].append(1)
                    dic['Phrase'].append(frase.upper())
            # except:
            #     errorsDic.append(file)
            #     pass
        else:
            print('File not in list')
print("--- %s seconds ---" % (time.time() - start_time))

File not in list
File not in list
File not in list
File not in list
File not in list
Bando_BCS_Comondu.pdf
Bando_BCS_LaPaz.pdf


KeyboardInterrupt: 

In [8]:
# dic
df = pd.DataFrame(dic)
df
# errorsDF = pd.DataFrame(errorsDic)

,Title,Page,Phrase


In [ ]:
df.to_csv('./Results/Bandos Resultados Tesseract.csv', index=False, encoding= 'utf-8-sig')
errorsDF.to_csv('./Results/Errores Bandos Tesseract.csv', index=False, encoding= 'utf-8-sig')